In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Code LLaMA

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a> (A Python-3 CPU notebook is recommended)
  </td>
</table>

## Overview

This notebook showcases deploying pretrained Code LLaMA models using a HuggingFace transformers based serving container and using [vLLM](https://github.com/vllm-project/vllm). This notebook also demonstrates how to evaluate the Code LLaMA models using EleutherAI's Language Model Evaluation Harness (lm-evaluation-harness) with Vertex CustomJob.

### Objective

- Deploy pre-trained Code LLaMA models using a standard HuggingFace serving solution
- Deploy pre-trained Code LLaMA models with [vLLM](https://github.com/vllm-project/vllm) with best serving throughput
- Evaluate Code LLaMA models

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Compute Engine API and Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,language.googleapis.com).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

Set the following variables for the experiment environment. The specified Cloud Storage bucket (BUCKET_URI) should be located in the specified region (REGION). Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1").

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
GCS_BUCKET = "gs://"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(GCS_BUCKET, "staging")
MODEL_BUCKET = os.path.join(GCS_BUCKET, "code-llama")

# The service account ends with:
# '.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### Initialize Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker images.
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-serve:20231026_1907_RC00"
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231002_0916_RC00"
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20231011_0934_RC00"

### Define common functions

In [ ]:
import os
from datetime import datetime

from google.cloud import aiplatform


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name: str,
    model_id: str,
    finetuned_lora_model_path: str,
    service_account: str,
    task: str,
    precision_loading_mode: str = "float16",
    machine_type: str = "n1-standard-8",
    accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_count: int = 1,
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "PRECISION_LOADING_MODE": precision_loading_mode,
        "TASK": task,
    }
    if finetuned_lora_model_path:
        serving_env["FINETUNED_LORA_MODEL_PATH"] = finetuned_lora_model_path
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=PREDICTION_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/peft_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    precision: str = "float16",
    swap_space: int = 16,
    machine_type: str = "n1-standard-8",
    accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_count: int = 1,
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        f"--swap-space={swap_space}",
        "--gpu-memory-utilization=0.9",
        "--max-num-batched-tokens=16385",
        f"--dtype={precision}",
        "--disable-log-stats",
    ]
    serving_env = {
        "MODEL_ID": "meta/code-llama@001",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

## Access pretrained Code LLaMA models
The original models from Meta are converted into the HuggingFace format for serving in Vertex AI.

Accept the model agreement to access the models:
1. Navigate to the Vertex AI > Model Garden page in the Google Cloud console
2. Find the [Code LLaMA model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/137) and click on "VIEW DETAILS"
3. Review the agreement on the model card page
4. After clicking the agreement of Code LLaMA, a Cloud Storage bucket containing Code LLaMA pretrained and finetuned models will be shared


In [ ]:
# This path will be shared once click the agreement in Code LLaMA model card
# as described in the `Access pretrained Code LLaMA models` section.
VERTEX_AI_MODEL_GARDEN_CODE_LLAMA = ""
assert (
    VERTEX_AI_MODEL_GARDEN_CODE_LLAMA
), "Please click the agreement of Code LLaMA in Vertex AI Model Garden, and get the GCS path of Code LLaMA model artifacts."
print(
    "Copying Code LLaMA model artifacts from",
    VERTEX_AI_MODEL_GARDEN_CODE_LLAMA,
    "to ",
    MODEL_BUCKET,
)
! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_CODE_LLAMA/* $MODEL_BUCKET

Select the model.

In [ ]:
model_name = "CodeLlama-7b-Instruct-hf"  # @param ["CodeLlama-7b-hf", "CodeLlama-7b-Python-hf", "CodeLlama-7b-Instruct-hf", "CodeLlama-13b-hf", "CodeLlama-13b-Python-hf", "CodeLlama-13b-Instruct-hf", "CodeLlama-34b-hf", "CodeLlama-34b-Python-hf", "CodeLlama-34b-Instruct-hf", "CodeLlama-70b-hf", "CodeLlama-70b-Python-hf", "CodeLlama-70b-Instruct-hf"]
model_id = os.path.join(MODEL_BUCKET, model_name)
print(model_id)

## Deploy pretrained Code LLaMA

This section deploys prebuilt Code LLaMA models on Vertex AI. V100 GPUs are used for demonstration. Note that V100 serving generally offers better throughput and latency performance than L4 serving, while L4 serving is generally more cost efficient than V100 serving. The serving efficiency of V100 and L4 GPUs is inferior to that of A100 GPUs, but V100 and L4 GPUs are nevertheless good serving solutions if you do not have A100 quota.

We use the PEFT serving image to deploy prebuilt Code LLaMA models, by setting finetuning LoRA model paths as empty. The model deployment step will take 15 minutes to 1 hour to complete, depending on the model sizes.

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 1 V100 (16G) to deploy 7B models.
# V100 serving has better throughput and latency performance than L4 serving.
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1

# Sets 1 L4 (24G) to deploy 7B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# Sets A100 (40G) to deploy 7B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 2 V100 (16G) to deploy 13B models.
# V100 serving has better throughput and latency performance than L4 serving.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2

# Sets 2 L4 (24G) to deploy 13B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

# Sets A100 (40G) to deploy 13B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 8 V100 (16G) to deploy 34B models.
# V100 serving has better throughput and latency performance than L4 serving.
# machine_type = "n1-standard-32"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 8

# Sets 4 L4 (24G) to deploy 34B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Sets 2 A100 (40G) to deploy 34B models.
# machine_type = "a2-highgpu-2g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 2

# Sets 8 L4 (24G) to deploy 70B models.
# machine_type = "g2-standard-96"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 8

# Sets 4 A100 (40G) to deploy 70B models.
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4

precision_loading_mode = "float16"

model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="code-llama-serve-peft"),
    model_id=model_id,
    finetuned_lora_model_path="",  # This will avoid override finetuning models.
    service_account=SERVICE_ACCOUNT,
    task="causal-language-modeling-lora",
    precision_loading_mode=precision_loading_mode,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print(f"Endpoint name: {endpoint.name}")

NOTE: The prebuilt model weights will be downloaded on the fly from the $MODEL_BUCKET after the deployment succeeds. Thus additional 10 ~ 40 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "prompt": "import argparse",
        "max_tokens": 200,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)
print(response.predictions[0])

## Deploy Pretrained Code LLaMA with vLLM

This section deploys prebuilt Code LLaMA models with [vLLM](https://github.com/vllm-project/vllm) on the Endpoint. Code LLaMA model weights are stored in bfloat16 precision. L4 or A100 GPUs are needed for vLLM serving at bfloat16 precision. V100 GPUs can be used with vLLM serving at float16 precision. Changing the precision from bfloat16 to float16 can result in a change in performance, and this change can be an increase and a decrease. However, the performance change should be small (within 5%).

V100 GPUs are used for demonstration. Note that V100 serving generally offers better throughput and latency performance than L4 serving, while L4 serving is generally more cost efficient than V100 serving. The serving efficiency of V100 and L4 GPUs is inferior to that of A100 GPUs, but V100 and L4 GPUs are nevertheless good serving solutions if you do not have A100 quota. The model deployment step will take 15 minutes to 1 hour to complete.

The vLLM project is an highly optimized LLM serving framework which can increase serving throughput a lot. The higher QPS you have, the more benefits you get using vLLM.

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 1 V100 (16G) to deploy 7B models.
# V100 serving has better throughput and latency performance than L4 serving.
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1
vllm_precision = "float16"
vllm_swap_space = 16

# Sets 1 L4 (24G) to deploy 7B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets A100 (40G) to deploy 7B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets 2 V100 (16G) to deploy 13B models.
# V100 serving has better throughput and latency performance than L4 serving.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2
# vllm_precision = "float16"
# vllm_swap_space = 16

# Sets 2 L4 (24G) to deploy 13B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets A100 (40G) to deploy 13B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets 8 V100 (16G) to deploy 34B models.
# V100 serving has better throughput and latency performance than L4 serving.
# machine_type = "n1-highmem-32"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 8
# vllm_precision = "float16"
# vllm_swap_space = 12

# Sets 4 L4 (24G) to deploy 34B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets 2 A100 (40G) to deploy 34B models.
# machine_type = "a2-highgpu-2g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 2
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets 8 L4 (24G) to deploy 70B models.
# machine_type = "g2-standard-96"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 8
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

# Sets 4 A100 (40G) to deploy 70B models.
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4
# vllm_precision = "bfloat16"
# vllm_swap_space = 16

model_vllm, endpoint_vllm = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="code-llama-serve-vllm"),
    model_id=model_id,
    service_account=SERVICE_ACCOUNT,
    precision=vllm_precision,
    swap_space=vllm_swap_space,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print(f"Endpoint name: {endpoint_vllm.name}")

NOTE: The prebuilt model weights will be downloaded on the fly from the $MODEL_BUCKET after the deployment succeeds. Thus additional 10 ~ 40 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts. Parameters supported by vLLM can be found [here](https://github.com/vllm-project/vllm/blob/2e8e49fce3775e7704d413b2f02da6d7c99525c9/vllm/sampling_params.py#L23-L64).

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_vllm.name` allows us to get the endpoint
#   name of the endpoint `endpoint_without_peft` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_vllm.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint_vllm = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "prompt": "import argparse",
        "n": 1,
        "max_tokens": 200,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_vllm.predict(instances=instances)
print(response.predictions[0])

## Evaluate PEFT-finetuned Code LLaMA models

This section demonstrates how to evaluate the Code LLaMA models using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex CustomJob. Please reference the peak GPU memory usgaes for serving and adjust the machine type, accelerator type and accelerator count accordingly.

This example uses the dataset [GSM8K](https://arxiv.org/abs/2110.14168). All supported tasks are listed in [this task table](https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md).


In [ ]:
eval_dataset = "gsm8k"  # @param {type:"string"}

In [ ]:
# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute

# Sets 1 V100 (16G) to evaluate 7B models.
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1

# Sets 1 L4 (24G) to evaluate 7B models.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# Sets A100 (40G) to evaluate 7B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 2 V100 (16G) to evaluate 13B models.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2

# Sets 2 L4 (24G) to evaluate 13B models.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

# Sets A100 (40G) to evaluate 13B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 8 V100 (16G) to evaluate 34B models.
machine_type = "n1-standard-32"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 8

# Sets 4 L4 (24G) to evaluate 34B models.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Sets 2 A100 (40G) to evaluate 34B models.
# machine_type = "a2-highgpu-2g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 2

# Sets 8 L4 (24G) to evaluate 70B models.
# machine_type = "g2-standard-96"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 8

# Sets 4 A100 (40G) to evaluate 70B models.
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4

replica_count = 1

In [ ]:
# Setup evaluation job.
job_name = get_job_name_with_datetime(prefix="code-llama-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")
model_id_gcsfuse = model_id.replace("gs://", "/gcs/")

In [ ]:
# Prepare evaluation command that runs the evaluation harness.
# Set `trust_remote_code = True` because evaluating the model requires
# executing code from the model repository.
# Set `use_accelerate = True` to enable evaluation across multiple GPUs.
eval_command = [
    "python",
    "main.py",
    "--model",
    "hf-causal-experimental",
    "--model_args",
    f"pretrained={model_id_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    "--tasks",
    f"{eval_dataset}",
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
]

### Submit evaluation CustomJob

In [ ]:
# Pass evaluation arguments and launch job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "command": eval_command,
            "args": [],
        },
    }
]

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
)

eval_job.run()

print("Evaluation results were saved in:", eval_output_dir)

### Fetch and print evaluation results

In [ ]:
import json

from google.cloud import storage

In [ ]:
# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = GCS_BUCKET.split("gs://")[1]
bucket = storage_client.get_bucket(BUCKET_NAME)
RESULT_FILE_PATH = eval_output_dir[len(GCS_BUCKET) + 1 :]
blob = bucket.blob(RESULT_FILE_PATH)
raw_result = blob.download_as_string()

In [ ]:
# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

## Clean up resources

In [ ]:
# Undeploy models and delete endpoints.
endpoint.delete(force=True)
endpoint_vllm.delete(force=True)

# Delete models.
model.delete()
model_vllm.delete()

# Delete evaluation job.
eval_job.delete()

# Uncomment below to delete all artifacts
# !gsutil -m rm -r $STAGING_BUCKET $MODEL_BUCKET